### Logic :
- Merge the 3 tables to get an overview
- Extracted the IDs with null values for 'length' from Attribute table and called 'null_IDs'.
- Got the corresponding neighbors for each null_ID from the distance table.
- Got the length of all corresponding neighbors for each null_ID and average them.
- Imputed the average length of all neighbor_ID for missing length in Attribute table.
- Repeated steps above to for missing values of 'reading'
- Used KNN imputer to input remianing missing values.

In [1]:
# Libraries to do data manipulation
import numpy as np

import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

pd.options.display.float_format = '{:.2f}'.format

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
Attribute = pd.read_csv('Downloads/data/attributes.csv')
Index = pd.read_csv('Downloads/data/index.csv')
Distance = pd.read_csv('Downloads/data/distance.csv')

In [3]:
Attribute.head()

,ID,length,reading
0,210677047200000,NaN,0.00
1,210276756950000,190.85,1697.05
2,210886605850000,438.81,8040.55
3,211256520700000,NaN,0.00
4,211276664600000,418.29,8687.22


In [4]:
# Checking for no of null values in Attrubute table

Attribute.isnull().sum()

ID            0
length     5171
reading    2147
dtype: int64

In [5]:
Index.head()

,ID,area_index_1,area_index_2,area_index_3
0,212307073200000,88488b1037fffff,87488b11dffffff,86488b11fffffff
1,211256530050000,8826cbccdbfffff,8726cbccdffffff,8626cbccfffffff
2,211946817450000,8848d4cc5bfffff,8748d4cc5ffffff,8648d4cc7ffffff
3,211476698350000,8826c2d35bfffff,8726c2d35ffffff,8626c2d37ffffff
4,212496608900000,8826c9a1c9fffff,8726c9a1cffffff,8626c9a1fffffff


In [6]:
Distance.head()

,ID,neighbor_ID,distance
0,212506738400000,212506855950000,1458.90
1,211137007050000,211136959050000,2456.57
2,211946742750000,211946745400000,1621.58
3,210486700400000,212486839400000,2818.86
4,212075109700000,212076693800000,2689.08


In [7]:
Attribute.shape, Index.shape, Distance.shape

((17088, 3), (17088, 4), (985270, 3))

In [8]:
# Merging tables for analysis 

Att_Idx = pd.merge(Attribute, Index, on='ID')

Att_Idx.head()

,ID,length,reading,area_index_1,area_index_2,area_index_3
0,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff
1,210276756950000,190.85,1697.05,88489e85edfffff,87489e85effffff,86489e85fffffff
2,210886605850000,438.81,8040.55,8848916637fffff,874891663ffffff,864891667ffffff
3,211256520700000,NaN,0.00,8826cb5a49fffff,8726cb5a4ffffff,8626cb5a7ffffff
4,211276664600000,418.29,8687.22,884891325bfffff,874891ad6ffffff,864891ad7ffffff


In [9]:
Att_Idx.shape

(17088, 6)

In [10]:
# Merging tables for analysis

Att_Idx_Dis = pd.merge(Att_Idx, Distance, on='ID')

Att_Idx_Dis.head()

,ID,length,reading,area_index_1,area_index_2,area_index_3,neighbor_ID,distance
0,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677047000000,2153.65
1,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677185950000,2685.26
2,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677185900000,2685.26
3,210276756950000,190.85,1697.05,88489e85edfffff,87489e85effffff,86489e85fffffff,210276747150000,1936.35
4,210886605850000,438.81,8040.55,8848916637fffff,874891663ffffff,864891667ffffff,210886626950000,2312.59


In [11]:
Att_Idx_Dis.shape

(985270, 8)

In [12]:
# Creating table for only rows where column 'length' is null

Att_Idx_Dis_null_lenght = Att_Idx_Dis[Att_Idx_Dis['length'].isnull()]
Att_Idx_Dis_null_lenght.head()

,ID,length,reading,area_index_1,area_index_2,area_index_3,neighbor_ID,distance
0,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677047000000,2153.65
1,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677185950000,2685.26
2,210677047200000,NaN,0.00,8848d6ca93fffff,8748d6cf4ffffff,8648d6cf7ffffff,210677185900000,2685.26
6,211256520700000,NaN,0.00,8826cb5a49fffff,8726cb5a4ffffff,8626cb5a7ffffff,211256509850000,3277.93
31,210486704650000,NaN,1705.30,8826ce25edfffff,8726ce25effffff,8626ce25fffffff,210486705300000,1889.81


In [13]:
Att_Idx_Dis_null_lenght.shape

(271910, 8)

In [14]:
# Creating a table of ID with missing 'length' value and the corresponding neighbors IDs

null_id_neigh = Att_Idx_Dis_null_lenght[['ID','neighbor_ID']]
null_id_neigh

,ID,neighbor_ID
0,210677047200000,210677047000000
1,210677047200000,210677185950000
2,210677047200000,210677185900000
6,211256520700000,211256509850000
31,210486704650000,210486705300000
...,...,...
985264,210515012150000,210515012150000
985265,210996640450000,210996640450000
985266,210996640450000,210996655900000
985267,211776684700000,211776684700000


In [15]:
# Creating a dictionary of IDs and length for Attribute table

length_dict = dict(zip(Attribute['ID'], Attribute['length']))

In [16]:
# To get neighbor ID length

null_id_neigh['length'] = null_id_neigh['neighbor_ID'].map(length_dict)
null_id_neigh

,ID,neighbor_ID,length
0,210677047200000,210677047000000,NaN
1,210677047200000,210677185950000,64.89
2,210677047200000,210677185900000,72.77
6,211256520700000,211256509850000,211.66
31,210486704650000,210486705300000,NaN
...,...,...,...
985264,210515012150000,210515012150000,NaN
985265,210996640450000,210996640450000,NaN
985266,210996640450000,210996655900000,NaN
985267,211776684700000,211776684700000,NaN


In [17]:
# Renaming columns for better representation
null_id_neigh.rename(columns = {'ID':'null_ID', 'length':'neighbor_length'}, inplace = True)
null_id_neigh

,null_ID,neighbor_ID,neighbor_length
0,210677047200000,210677047000000,NaN
1,210677047200000,210677185950000,64.89
2,210677047200000,210677185900000,72.77
6,211256520700000,211256509850000,211.66
31,210486704650000,210486705300000,NaN
...,...,...,...
985264,210515012150000,210515012150000,NaN
985265,210996640450000,210996640450000,NaN
985266,210996640450000,210996655900000,NaN
985267,211776684700000,211776684700000,NaN


In [18]:
# Find sum of null values in dataframe

null_id_neigh.isna().sum()

null_ID                 0
neighbor_ID             0
neighbor_length    156098
dtype: int64

In [19]:
# Removing rows where neighbor_length is null as wont be useful for our evaluation

null_id_neigh = null_id_neigh[null_id_neigh['neighbor_length'].notnull()]
null_id_neigh

,null_ID,neighbor_ID,neighbor_length
1,210677047200000,210677185950000,64.89
2,210677047200000,210677185900000,72.77
6,211256520700000,211256509850000,211.66
38,211647204000000,211646989550000,332.80
82,211836704950000,211836761550000,709.84
...,...,...,...
985236,211826844850000,211826892550000,232.47
985237,211826844850000,211016751200000,458.09
985253,210636649050000,212536657850000,681.25
985258,210325052600000,210326608450000,197.31


In [20]:
null_id_neigh.describe().T

,count,mean,std,min,25%,50%,75%,max
null_ID,115812.00,211431464557528.88,582511736404.69,210016534050000.00,211136994750000.00,211276835650000.00,211916639350000.00,212536669700000.00
neighbor_ID,115812.00,211439466541227.19,577763281055.21,210016589950000.00,211176807000000.00,211276780650000.00,211856747400000.00,212536669550000.00
neighbor_length,115812.00,522.22,226.96,50.92,354.25,476.90,634.03,1571.11


In [21]:
# Finding the average of neighbor_ID length for the null_ID 

neighbor_avg_len = null_id_neigh.groupby('null_ID')['neighbor_length'].mean().to_frame().reset_index()
neighbor_avg_len.head()

,null_ID,neighbor_length
0,210016534050000,772.79
1,210016534200000,1127.27
2,210016545550000,784.95
3,210016837200000,736.16
4,210016847300000,532.76


In [22]:
# Creating new Attribute table

Attribute_null = Attribute.copy()
Attribute_null.drop('reading', axis = 1, inplace = True)
Attribute_null

,ID,length
0,210677047200000,NaN
1,210276756950000,190.85
2,210886605850000,438.81
3,211256520700000,NaN
4,211276664600000,418.29
...,...,...
17083,210515012150000,NaN
17084,210996640450000,NaN
17085,211776684700000,NaN
17086,212526829000000,NaN


In [23]:
# Mapping ID with missing 'length' values in Attribute table to corresponding average neighbor_ID lengths  

Attribute_new = (Attribute_null.replace('NaN', float('nan'))
    .set_index(['ID']).T
    .fillna(neighbor_avg_len.set_index(['null_ID'])['neighbor_length'])
    .T.reset_index()
 )
Attribute_new

,ID,length
0,210677047200000,68.83
1,210276756950000,190.85
2,210886605850000,438.81
3,211256520700000,211.66
4,211276664600000,418.29
...,...,...
17083,210515012150000,NaN
17084,210996640450000,NaN
17085,211776684700000,NaN
17086,212526829000000,NaN


**To Get Null Values for 'reading' column**

In [24]:
# Creating table for only rows where column 'reading' is null

Att_Idx_Dis_null_reading = Att_Idx_Dis[Att_Idx_Dis['reading'].isnull()]
Att_Idx_Dis_null_reading.head()

,ID,length,reading,area_index_1,area_index_2,area_index_3,neighbor_ID,distance
35,211647204000000,NaN,NaN,88488b3adbfffff,87488b31effffff,86488b31fffffff,211647203050000,1676.02
36,211647204000000,NaN,NaN,88488b3adbfffff,87488b31effffff,86488b31fffffff,211647203150000,1676.04
37,211647204000000,NaN,NaN,88488b3adbfffff,87488b31effffff,86488b31fffffff,211647203450000,1676.57
38,211647204000000,NaN,NaN,88488b3adbfffff,87488b31effffff,86488b31fffffff,211646989550000,2922.75
39,211946817450000,492.00,NaN,8848d4cc5bfffff,8748d4cc5ffffff,8648d4cc7ffffff,211946928000000,1969.10


In [25]:
Att_Idx_Dis_null_reading.shape

(103831, 8)

In [26]:
# Creating a table of ID with missing 'reading' value and the corresponding neighbors IDs

null_id_neigh_r = Att_Idx_Dis_null_reading[['ID','neighbor_ID']]
null_id_neigh_r

,ID,neighbor_ID
35,211647204000000,211647203050000
36,211647204000000,211647203150000
37,211647204000000,211647203450000
38,211647204000000,211646989550000
39,211946817450000,211946928000000
...,...,...
985252,212386536300000,212386524450000
985253,210636649050000,212536657850000
985254,210636649050000,210636649050000
985255,210636649050000,210636652600000


In [27]:
# Creating a dictionary of IDs and reading for Attribute table

reading_dict = dict(zip(Attribute['ID'], Attribute['reading']))

In [28]:
null_id_neigh_r['reading'] = null_id_neigh_r['neighbor_ID'].map(reading_dict)
null_id_neigh_r

,ID,neighbor_ID,reading
35,211647204000000,211647203050000,0.00
36,211647204000000,211647203150000,0.00
37,211647204000000,211647203450000,0.00
38,211647204000000,211646989550000,749.58
39,211946817450000,211946928000000,5880.60
...,...,...,...
985252,212386536300000,212386524450000,NaN
985253,210636649050000,212536657850000,4868.30
985254,210636649050000,210636649050000,NaN
985255,210636649050000,210636652600000,NaN


In [29]:
# Renaming columns for better representation
null_id_neigh_r.rename(columns = {'ID':'null_ID', 'reading':'neighbor_reading'}, inplace = True)
null_id_neigh_r

,null_ID,neighbor_ID,neighbor_reading
35,211647204000000,211647203050000,0.00
36,211647204000000,211647203150000,0.00
37,211647204000000,211647203450000,0.00
38,211647204000000,211646989550000,749.58
39,211946817450000,211946928000000,5880.60
...,...,...,...
985252,212386536300000,212386524450000,NaN
985253,210636649050000,212536657850000,4868.30
985254,210636649050000,210636649050000,NaN
985255,210636649050000,210636652600000,NaN


In [30]:
# Find sum of null values in dataframe

null_id_neigh_r.isna().sum()

null_ID                 0
neighbor_ID             0
neighbor_reading    19997
dtype: int64

In [31]:
# Removing rows where neighbor_reading is null as wont be useful for our evaluation

null_id_neigh_r = null_id_neigh_r[null_id_neigh_r['neighbor_reading'].notnull()]
null_id_neigh_r

,null_ID,neighbor_ID,neighbor_reading
35,211647204000000,211647203050000,0.00
36,211647204000000,211647203150000,0.00
37,211647204000000,211647203450000,0.00
38,211647204000000,211646989550000,749.58
39,211946817450000,211946928000000,5880.60
...,...,...,...
985204,212486865250000,212486889500000,475.20
985205,212486865250000,211686761950000,1590.20
985250,212386536300000,212386535400000,1233.50
985251,212386536300000,211436625200000,848.70


In [32]:
null_id_neigh_r.describe().T

,count,mean,std,min,25%,50%,75%,max
null_ID,83834.00,211323490245307.25,566000822783.93,210016589950000.00,210886605600000.00,211276837700000.00,211647118450000.00,212536669700000.00
neighbor_ID,83834.00,211328057369161.25,561629888698.90,210015076400000.00,210886636500000.00,211276772700000.00,211647006700000.00,212536669050000.00
neighbor_reading,83834.00,7037.22,6037.66,0.00,2489.30,6073.24,10225.25,67531.48


In [33]:
# Finding the average of neighbor_ID length for the null_ID 

neighbor_avg_reading = null_id_neigh_r.groupby('null_ID')['neighbor_reading'].mean().to_frame().reset_index()
neighbor_avg_reading.head()

,null_ID,neighbor_reading
0,210016589950000,375.42
1,210016953700000,1569.84
2,210016954950000,181.36
3,210017133900000,519.37
4,210017133950000,519.37


In [34]:
# Creating new Attribute table

Attribute_null_r = Attribute.copy()
Attribute_null_r.drop('length', axis = 1, inplace = True)
Attribute_null_r

,ID,reading
0,210677047200000,0.00
1,210276756950000,1697.05
2,210886605850000,8040.55
3,211256520700000,0.00
4,211276664600000,8687.22
...,...,...
17083,210515012150000,0.00
17084,210996640450000,3329.40
17085,211776684700000,1563.00
17086,212526829000000,230.97


In [35]:
# Mapping ID with missing 'reading' values in Attribute table to corresponding average neighbor_ID reading  

Attribute_new_r = (Attribute_null_r.replace('NaN', float('nan'))
    .set_index(['ID']).T
    .fillna(neighbor_avg_reading.set_index(['null_ID'])['neighbor_reading'])
    .T.reset_index()
 )
Attribute_new_r

,ID,reading
0,210677047200000,0.00
1,210276756950000,1697.05
2,210886605850000,8040.55
3,211256520700000,0.00
4,211276664600000,8687.22
...,...,...
17083,210515012150000,0.00
17084,210996640450000,3329.40
17085,211776684700000,1563.00
17086,212526829000000,230.97


In [36]:
Attribute_new['reading'] = Attribute_new_r['reading']
Attribute_new

,ID,length,reading
0,210677047200000,68.83,0.00
1,210276756950000,190.85,1697.05
2,210886605850000,438.81,8040.55
3,211256520700000,211.66,0.00
4,211276664600000,418.29,8687.22
...,...,...,...
17083,210515012150000,NaN,0.00
17084,210996640450000,NaN,3329.40
17085,211776684700000,NaN,1563.00
17086,212526829000000,NaN,230.97


In [37]:
# Finding missing values

Attribute_new.isna().sum()

ID            0
length     1164
reading     119
dtype: int64

**Applying KNN imputation for remainig misiing values**

In [38]:
# Drop ID column as it is all unique 
Attribute_new.drop('ID', axis = 1, inplace = True)

In [39]:
imputer = KNNImputer(n_neighbors=3)
Attribute_imputed = imputer.fit_transform(Attribute_new)


In [40]:
Attribute_imputed_knn = pd.DataFrame(Attribute_imputed, columns=Attribute_new.columns)
Attribute_imputed_knn

,length,reading
0,68.83,0.00
1,190.85,1697.05
2,438.81,8040.55
3,211.66,0.00
4,418.29,8687.22
...,...,...
17083,299.66,0.00
17084,639.74,3329.40
17085,672.71,1563.00
17086,476.64,230.97


In [41]:
Attribute_imputed_knn.isna().sum()

length     0
reading    0
dtype: int64

In [42]:
Attribute_imputed_knn['ID'] = Attribute['ID']
Attribute_imputed_knn = Attribute_imputed_knn.set_index(['ID'])
Attribute_imputed_knn

,length,reading
ID,,
210677047200000,68.83,0.00
210276756950000,190.85,1697.05
210886605850000,438.81,8040.55
211256520700000,211.66,0.00
211276664600000,418.29,8687.22
...,...,...
210515012150000,299.66,0.00
210996640450000,639.74,3329.40
211776684700000,672.71,1563.00


**Trying Iterative Inputter** 

In [43]:
Itr_imputer = IterativeImputer()
Attribute_imputed_I = Itr_imputer.fit_transform(Attribute_new)


In [44]:
Attribute_imputed_Itr = pd.DataFrame(Attribute_imputed_I, columns=Attribute_new.columns)
Attribute_imputed_Itr


,length,reading
0,68.83,0.00
1,190.85,1697.05
2,438.81,8040.55
3,211.66,0.00
4,418.29,8687.22
...,...,...
17083,493.11,0.00
17084,549.54,3329.40
17085,519.60,1563.00
17086,497.02,230.97


In [45]:
Attribute_imputed_Itr.isna().sum()


length     0
reading    0
dtype: int64

In [46]:
Attribute_imputed_Itr['ID'] = Attribute['ID']
Attribute_imputed_Itr = Attribute_imputed_Itr.set_index(['ID'])
Attribute_imputed_Itr

,length,reading
ID,,
210677047200000,68.83,0.00
210276756950000,190.85,1697.05
210886605850000,438.81,8040.55
211256520700000,211.66,0.00
211276664600000,418.29,8687.22
...,...,...
210515012150000,493.11,0.00
210996640450000,549.54,3329.40
211776684700000,519.60,1563.00


**Conclusion:**
- **Attribute_imputed_knn** is the final table using KNN imputer which is most common and prefered method.
- Attribute_imputed_itr is the final table using iterative imputer - experimental.